In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Parsing



In [1]:
!pip install demoparser2
!pip install pandas numpy scipy scikit-learn

In [6]:
import os
import demoparser2
import pandas as pd
from demoparser2 import DemoParser
from tqdm import tqdm

Parse Per Demo

In [4]:
def parse_single_demo_file(file_path, label, output_dir):
  parser = DemoParser(file_path)
  player_death_events = parser.parse_event("player_death")
  df = parser.parse_ticks(["pitch", "yaw", "tick", "steamid"])
  df['label'] = label
  base_dir = os.path.join(output_dir, "cheater" if label == 1 else "legit")
  os.makedirs(base_dir, exist_ok=True)
  for idx, event in tqdm(player_death_events.iterrows(), total=player_death_events.shape[0], desc=f"Parsing {os.path.basename(file_path)}"):
    start_tick = event["tick"] - 300
    end_tick = event["tick"]
    attacker = event.get("attacker_steamid")
    if attacker is not None:
      attacker_int = int(attacker)
      subdf = df[(df["tick"].between(start_tick, end_tick)) & (df["steamid"] == attacker_int)]
      if not subdf.empty:
        user_dir = os.path.join(base_dir, f"user_{attacker}")
        os.makedirs(user_dir, exist_ok=True)
        filename = f"kill_event_{start_tick}_to_{end_tick}.csv"
        file_path = os.path.join(user_dir, filename)
        subdf.to_csv(file_path, index=False)
        print(f"Data saved to {file_path}")

Parse Per Folder

In [7]:
def parse_demo_folder(input_dir, label, output_dir):
    subfolder = 'cheater' if label == 1 else 'legit'
    base_dir = os.path.join(output_dir, subfolder)
    os.makedirs(base_dir, exist_ok=True)

    for filename in os.listdir(input_dir):
        if not filename.endswith('.dem'):
            continue
        demo_path = os.path.join(input_dir, filename)
        demo_base = os.path.splitext(filename)[0]

        # Skip if we’ve already parsed this demo
        already = [f for f in os.listdir(base_dir) if f.startswith(demo_base)]
        if already:
            print(f"Skipping {filename}, already parsed.")
            continue

        print(f"Parsing {filename}…")
        parser = DemoParser(demo_path)
        events = parser.parse_event("player_death")
        df = parser.parse_ticks(["pitch", "yaw", "tick", "steamid"])
        df['label'] = label

        for _, event in events.iterrows():
            start_tick = event["tick"] - 300
            end_tick   = event["tick"]
            attacker   = event.get("attacker_steamid")
            if not attacker:
                continue

            attacker_int = int(attacker)
            window = df[
                df["tick"].between(start_tick, end_tick) &
                (df["steamid"] == attacker_int)
            ]
            if window.empty:
                continue

            # 1) Drop any duplicate tick rows
            window = window.drop_duplicates(subset="tick")
            # 2) Reindex so we have exactly start_tick…end_tick‑1
            full_index = list(range(start_tick, end_tick))
            window = (window
                      .set_index("tick")
                      .reindex(full_index)
                      .fillna(method="ffill")  # carry last known pitch/yaw
                      .reset_index()
                      .rename(columns={"index": "tick"}))
            window['steamid'] = attacker_int
            window['label']   = label

            # Save
            user_dir = os.path.join(base_dir, f"user_{attacker}")
            os.makedirs(user_dir, exist_ok=True)
            csv_name  = f"{demo_base}_kill_{start_tick}_to_{end_tick}.csv"
            csv_path  = os.path.join(user_dir, csv_name)
            window.to_csv(csv_path, index=False)
            print(f"  → Saved {csv_path}")

In [8]:
cheater_dir = '../data/raw/demos/cheater'
legit_dir = '../data/raw/demos/legit'
output_dir = '../data/interim/parsed_csv'

In [9]:
#print("Parsing cheater demos...")
#parse_demo_folder(cheater_dir, label=1, output_dir=output_dir)

print("Parsing legit demos...")
parse_demo_folder(legit_dir, label=0, output_dir=output_dir)

Parsing legit demos...
Parsing sarkreez3.dem…


/tmp/ipykernel_8457/1191751963.py:46: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  .fillna(method="ffill")  # carry last known pitch/yaw
/tmp/ipykernel_8457/1191751963.py:46: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  .fillna(method="ffill")  # carry last known pitch/yaw
/tmp/ipykernel_8457/1191751963.py:46: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  .fillna(method="ffill")  # carry last known pitch/yaw
/tmp/ipykernel_8457/1191751963.py:46: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  .fillna(method="ffill")  # carry last known pitch/yaw
/tmp/ipykernel_8457/1191751963.py:46: FutureWarning: DataFrame.fillna with 'meth

  → Saved ../data/interim/parsed_csv/legit/user_76561198378606016/sarkreez3_kill_2466_to_2766.csv
  → Saved ../data/interim/parsed_csv/legit/user_76561198380850569/sarkreez3_kill_2692_to_2992.csv
  → Saved ../data/interim/parsed_csv/legit/user_76561198084634875/sarkreez3_kill_3242_to_3542.csv
  → Saved ../data/interim/parsed_csv/legit/user_76561198349713530/sarkreez3_kill_3262_to_3562.csv
  → Saved ../data/interim/parsed_csv/legit/user_76561199019469674/sarkreez3_kill_3805_to_4105.csv
  → Saved ../data/interim/parsed_csv/legit/user_76561198986519110/sarkreez3_kill_4674_to_4974.csv
  → Saved ../data/interim/parsed_csv/legit/user_76561199019469674/sarkreez3_kill_4897_to_5197.csv
  → Saved ../data/interim/parsed_csv/legit/user_76561199019469674/sarkreez3_kill_7151_to_7451.csv
  → Saved ../data/interim/parsed_csv/legit/user_76561199019469674/sarkreez3_kill_7407_to_7707.csv
  → Saved ../data/interim/parsed_csv/legit/user_76561198378606016/sarkreez3_kill_7480_to_7780.csv
  → Saved ../data/in

/tmp/ipykernel_8457/1191751963.py:46: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  .fillna(method="ffill")  # carry last known pitch/yaw
/tmp/ipykernel_8457/1191751963.py:46: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  .fillna(method="ffill")  # carry last known pitch/yaw
/tmp/ipykernel_8457/1191751963.py:46: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  .fillna(method="ffill")  # carry last known pitch/yaw
/tmp/ipykernel_8457/1191751963.py:46: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  .fillna(method="ffill")  # carry last known pitch/yaw
/tmp/ipykernel_8457/1191751963.py:46: FutureWarning: DataFrame.fillna with 'meth

  → Saved ../data/interim/parsed_csv/legit/user_76561199019469674/sarkreez3_kill_51179_to_51479.csv
  → Saved ../data/interim/parsed_csv/legit/user_76561198349713530/sarkreez3_kill_54297_to_54597.csv
  → Saved ../data/interim/parsed_csv/legit/user_76561198380850569/sarkreez3_kill_55192_to_55492.csv
  → Saved ../data/interim/parsed_csv/legit/user_76561199019469674/sarkreez3_kill_55562_to_55862.csv
  → Saved ../data/interim/parsed_csv/legit/user_76561199019469674/sarkreez3_kill_55637_to_55937.csv
  → Saved ../data/interim/parsed_csv/legit/user_76561198349713530/sarkreez3_kill_55821_to_56121.csv
  → Saved ../data/interim/parsed_csv/legit/user_76561199019469674/sarkreez3_kill_55828_to_56128.csv
  → Saved ../data/interim/parsed_csv/legit/user_76561199055234585/sarkreez3_kill_56014_to_56314.csv
  → Saved ../data/interim/parsed_csv/legit/user_76561198378606016/sarkreez3_kill_56268_to_56568.csv
  → Saved ../data/interim/parsed_csv/legit/user_76561198349713530/sarkreez3_kill_58825_to_59125.csv


...

/tmp/ipykernel_8457/1191751963.py:46: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  .fillna(method="ffill")  # carry last known pitch/yaw
/tmp/ipykernel_8457/1191751963.py:46: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  .fillna(method="ffill")  # carry last known pitch/yaw
/tmp/ipykernel_8457/1191751963.py:46: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  .fillna(method="ffill")  # carry last known pitch/yaw
/tmp/ipykernel_8457/1191751963.py:46: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  .fillna(method="ffill")  # carry last known pitch/yaw
/tmp/ipykernel_8457/1191751963.py:46: FutureWarning: DataFrame.fillna with 'meth

  → Saved ../data/interim/parsed_csv/legit/user_76561198051101287/sarkreez1_kill_2246_to_2546.csv
  → Saved ../data/interim/parsed_csv/legit/user_76561198051101287/sarkreez1_kill_2323_to_2623.csv
  → Saved ../data/interim/parsed_csv/legit/user_76561198119610844/sarkreez1_kill_2326_to_2626.csv
  → Saved ../data/interim/parsed_csv/legit/user_76561198373660142/sarkreez1_kill_2607_to_2907.csv
  → Saved ../data/interim/parsed_csv/legit/user_76561198832363808/sarkreez1_kill_3166_to_3466.csv
  → Saved ../data/interim/parsed_csv/legit/user_76561198402911368/sarkreez1_kill_3214_to_3514.csv
  → Saved ../data/interim/parsed_csv/legit/user_76561199124095891/sarkreez1_kill_3298_to_3598.csv
  → Saved ../data/interim/parsed_csv/legit/user_76561198402911368/sarkreez1_kill_5976_to_6276.csv
  → Saved ../data/interim/parsed_csv/legit/user_76561198286419199/sarkreez1_kill_6008_to_6308.csv
  → Saved ../data/interim/parsed_csv/legit/user_76561198200469473/sarkreez1_kill_6841_to_7141.csv
  → Saved ../data/in

/tmp/ipykernel_8457/1191751963.py:46: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  .fillna(method="ffill")  # carry last known pitch/yaw
/tmp/ipykernel_8457/1191751963.py:46: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  .fillna(method="ffill")  # carry last known pitch/yaw
/tmp/ipykernel_8457/1191751963.py:46: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  .fillna(method="ffill")  # carry last known pitch/yaw
/tmp/ipykernel_8457/1191751963.py:46: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  .fillna(method="ffill")  # carry last known pitch/yaw
/tmp/ipykernel_8457/1191751963.py:46: FutureWarning: DataFrame.fillna with 'meth

  → Saved ../data/interim/parsed_csv/legit/user_76561198402911368/sarkreez1_kill_50267_to_50567.csv
  → Saved ../data/interim/parsed_csv/legit/user_76561198373660142/sarkreez1_kill_50331_to_50631.csv
  → Saved ../data/interim/parsed_csv/legit/user_76561198286419199/sarkreez1_kill_53546_to_53846.csv
  → Saved ../data/interim/parsed_csv/legit/user_76561199124095891/sarkreez1_kill_53601_to_53901.csv
  → Saved ../data/interim/parsed_csv/legit/user_76561198200469473/sarkreez1_kill_53634_to_53934.csv
  → Saved ../data/interim/parsed_csv/legit/user_76561198931708958/sarkreez1_kill_53716_to_54016.csv
  → Saved ../data/interim/parsed_csv/legit/user_76561198119610844/sarkreez1_kill_53853_to_54153.csv
  → Saved ../data/interim/parsed_csv/legit/user_76561198931708958/sarkreez1_kill_54485_to_54785.csv
  → Saved ../data/interim/parsed_csv/legit/user_76561198119610844/sarkreez1_kill_54899_to_55199.csv
  → Saved ../data/interim/parsed_csv/legit/user_76561198119610844/sarkreez1_kill_55088_to_55388.csv


/tmp/ipykernel_8457/1191751963.py:46: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  .fillna(method="ffill")  # carry last known pitch/yaw
/tmp/ipykernel_8457/1191751963.py:46: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  .fillna(method="ffill")  # carry last known pitch/yaw
/tmp/ipykernel_8457/1191751963.py:46: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  .fillna(method="ffill")  # carry last known pitch/yaw
/tmp/ipykernel_8457/1191751963.py:46: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  .fillna(method="ffill")  # carry last known pitch/yaw
/tmp/ipykernel_8457/1191751963.py:46: FutureWarning: DataFrame.fillna with 'meth